# Bibliotecas

In [ ]:
import cv2
import numpy as np
from keras.models import load_model

# Ativar WebCam e processamento 

In [ ]:
#tensorFlow e keras

video = cv2.VideoCapture(1,cv2.CAP_DSHOW) #qual tipo de camera sera utilizada

model = load_model('Keras_model.h5',compile=False) #utilizar para reconhecer a imagem
data = np.ndarray(shape=(1,224,224,3),dtype=np.float32) #definir o tipo de dado
classes = ["1 real","25 cent","50 cent"] #array que recebe as labels (indices)

#objetivo de contornar o objeto
def preProcess(img):
    imgPre = cv2.GaussianBlur(img,(5,5),3) #expandir os pixels da imagem(borrar imagem)
    imgPre = cv2.Canny(imgPre,90,140)#filtrar a imagem pelas bordas
    kernel = np.ones((4,4),np.uint8)#extrair bordas da imagem
    imgPre = cv2.dilate(imgPre,kernel,iterations=4) #dilatar a imagem
    imgPre = cv2.erode(imgPre,kernel,iterations=3) #erosão na imagem
    return imgPre

#reconhecimento de moedas 
def DetectarMoeda(img):
    imgMoeda = cv2.resize(img,(224,224))#redimencionar a imagem com tamamnho
    imgMoeda = np.asarray(imgMoeda)
    imgMoedaNormalize = (imgMoeda.astype(np.float32)/127.0)-1 #normalizar a imagem
    data[0] = imgMoedaNormalize 
    prediction = model.predict(data)#passo a imagem para o modelo
    index = np.argmax(prediction) #extrair o maior valor
    percent = prediction[0][index] #percentual de predição
    classe = classes[index]
    return classe,percent

#pegar a imagem cortada pre processar e descobrir qual é a moeda
while True:
    _,img = video.read()#leitura da imagem
    img = cv2.resize(img,(640,480))#redimensionar a imagem
    imgPre = preProcess(img) #chamando a imagem pre processada
    countors,hi = cv2.findContours(imgPre,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE) #rastrear os objetos
    
    if not _:
       print("sair do video")
       break

    qtd = 0 #contador
    for cnt in countors:  #percorrer para identificar cada objeto individual
        area = cv2.contourArea(cnt) #validar a area do objeto
        if area > 2000: #evitar ruido caso nao tenha moeda
            x,y,w,h = cv2.boundingRect(cnt) 
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)#definir retangulo ao encontrar um objeto
            recorte = img[y:y +h,x:x+ w] #recortar as moedas
            classe, conf = DetectarMoeda(recorte)#
            if conf >0.7: #ficar mais preciso o reconhecimento da imagem
                cv2.putText(img,str(classe),(x,y),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2)#visualizar qual é a classe
                if classe == '1 real': qtd+=1
                if classe == '25 cent': qtd += 0.25
                if classe == '50 cent': qtd += 0.5

    cv2.rectangle(img,(430,30),(600,80),(0,255,0),-1)#layout para mostrar valores
    cv2.putText(img,f'R$ {qtd}',(440,67),cv2.FONT_HERSHEY_SIMPLEX,1.2,(255,255,255),2) #mostrar valor

    cv2.imshow('IMG',img) #exibir imagem
    cv2.imshow('IMG PRE', imgPre)

    sair = cv2.waitKey(1) & 0xFF #checar se foi clicado na tela

    if sair == 27:  # Presionar esc para sair
        break
    
video.release()
cv2.destroyAllWindows()    